In [9]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import torch
import random

In [11]:
A = np.array(
    (
        (1, -1/2, 0, 0, 0, 0, 0, 0), 
        (1/2, -1, 1/2, 0, 0, 0, 0, 0),
        (0, 1/2, -1, 1/2, 0, 0, 0, 0),
        (0, 0, 1/3, -1, 1/3, 0, 0, 0),
        (0, 0, 0, 1/3, -1, 0, 1/3, 0),
        (0, 0, 0, 0, 0, -1, 1/2, 0),
        (0, 0, 0, 0, 1/3, 1/3, -1, 1/3),
        (0, 0, 0, 0, 0, 0, 1/2, -1)
    )
)

In [12]:
b = np.array((0, 0, 0, -1/3, 1/3, 0, 0, 1/2))

In [13]:
V = np.linalg.inv(A).dot(b)
V

array([ 0.06756757,  0.13513514,  0.2027027 ,  0.27027027, -0.39189189,
       -0.22297297, -0.44594595, -0.72297297])

In [14]:
V = np.array(
    (0, 0, 0, 0, 0, 0, 0, 0), dtype=float
)

r = np.array(
    (1, -10, -2, -1, 1), dtype=float 
)

for i in range(100):
    V[7] = .5*(V[6] - 1)
    V[6] = .33*(V[4] + V[5] + V[7])
    V[5] = .5*(0 + V[6])
    V[4] = .33*(V[3] + V[6] - 1)
    V[3] = .33*(V[2] + V[4] + 1)
    V[2] = .5*(V[1] + V[3])
    V[1] = .5*(V[0] + V[2])
    V[0] = .5*(0 + V[1])
V

array([ 0.06744991,  0.13489981,  0.20234972,  0.26979963, -0.38477509,
       -0.21789237, -0.43578474, -0.71789237])